# Introduction
Multi-instance (MI) machine learning approaches can be used to solve the issues of representation of each molecule by multiple conformations (instances) and automatic selection of the most relevant ones. In the multi-instance approach, an example (i.e., a molecule) is presented by a bag of instances (i.e., a set of conformations), and a label (a molecule property value) is available only for a bag (a molecule), but not for individual instances (conformations).

Here, we report an application of Multi-Instance Learning approach to predictive modeling of enantioselectivity of chiral catalysts. Catalysts were represented by ensembles of conformers encoded by the pmapper physicochemical descriptors capturing stereo configuration of the molecule. Each catalyzed chemical reaction was transformed to a Condensed Graph of Reaction for which ISIDA fragment descriptors were generated. This approach does not require any conformers’ alignment and can potentially be used for diverse set of catalysts bearing different scaffolds.

<img src="img/approach.png" width="500"/>

# Descriptors

Each reaction was transformed to a Condensed Graph of Reaction (CGR) with a CGRtools package. CGR is a single graph, which encodes an ensemble of reactants and products. CGR results from the superposition of the atoms of products and reactants having the same numbers. It contains both conventional chemical bonds (single, double, triple, aromatic, etc.) and so-called “dynamic” bonds describing chemical transformations, i.e. breaking or forming a bond or changing bond order. Given CGRs were encoded by ISIDA (In Silico Design and Data Analysis) fragment descriptors, counting the occurrence of particular subgraphs (structural fragments) of different topologies and sizes. In this study, atom-centered subgraphs containing a given atom with the atoms and bonds of its n coordination spheres (n = 1-4) are used.

<img src="img/cgr.png" width="500"/>

For each catalyst, up to 50 conformations (**nconfs**) within a 10 kcal/mol energy window (**energy**) have been generated using the distance geometry algorithm implemented in RDKit13. The conformations with RMSD values below 0.5Å with respect to selected conformers were removed in order to reduce redundancy. Then, selected conformers were encoded by a vector of pmapper descriptors. Each conformer is represented by an ensemble of physicochemical features assigned to atoms, functional groups, or rings: H-donor, H-acceptor, or hydrophobic, or positively or negatively charged. Rings are characterized by either hydrophobic or aromatic features. All possible combinations of features quadruplets are enumerated. Each quadruplet is encoded by a canonical signature, which contains information about comprising features, the distance between them, and stereoconfiguration. To enable fuzzy matching of quadruplets to identify similar ones, the distances between features are binned with the step of 1Å. Each unique quadruplet is considered as a descriptor whereas its count is a descriptor value. 
<img src="img/pmapper_descriptors.png" width="900"/>

Vectors of 2D fragment reaction descriptors and 3D physicochemical quadruplets were then concatenated to form combined reaction/catalyst descriptor vector. 
<img src="img/isida_pmapper_descriptors.png" width="800"/>

The descriptor calculation function reads an RDF file with reactions where the CATALYST_SMILES field contains the catalyst smiles. The SELECTIVITY field stores the experimental value of the selectivity (ΔΔG) in the reaction. The ID field contains a unique reaction index.

In [ ]:
import os
from miqssr.utils import calc_descriptors

input_fname = os.path.join('data', 'input_data.rdf')
nconfs = 50 # max number of conformers to generate
energy = 10 # energy window
ncpu = 20 # number of cpus
path = './descriptors' # where to store the calculated descriptors

out_fname = calc_descriptors(input_fname=input_fname, nconfs=nconfs, energy=energy, ncpu=ncpu, path=path)

# Model training
The descriptors file contains columns: *react_id* (reaction index), *mol_title* (reaction name), *act* - selectivity of reaction.

One should to implement a function to create a n × m × k list of bags (n - number of reactions, m - bag size (number of conformers generated), k - number of descriptors).

In [2]:
import numpy as np
import pandas as pd

def load_data(fname):
    #data = pd.read_csv(fname, index_col='react_id').sort_index()
    data = pd.read_csv(fname, index_col='mol_id').sort_index()
    bags, labels, idx = [], [], []
    for i in data.index.unique():
        bag = data.loc[i:i].drop(['mol_title', 'act'], axis=1).values
        label = float(data.loc[i:i]['act'].unique()[0])

        bags.append(bag)
        labels.append(label)
        idx.append(i)

    return np.array(bags), np.array(labels), idx


dsc_fname = os.path.join('descriptors', 'PhFprPmapper_concat-data_50.csv') # descriptors file
bags, labels, idx = load_data(dsc_fname)
print(f'There are {len(bags)} reactions encoded with {bags[0].shape[1]} descriptors')

Training set was 384 reactions (24 catalysts × 16 substrate combinations = 384 reactions), and the external test set was composed of the 691 reactions. 

In [34]:
def train_test_split_default(bags, labels, idx):
    
    test_reactions_idx = open('test_reactions.txt').read().split(',')

    x_train, x_test = [], []
    y_train, y_test = [], []
    idx_train, idx_test = [], []
    for bag, label, i in zip(bags, labels, idx):
        if i in test_idx:
            x_test.append(bag)
            y_test.append(label)
            idx_test.append(i)
        else:
            x_train.append(bag)
            y_train.append(label)
            idx_train.append(i)
            
    x_train, x_test, y_train, y_test = np.array(x_train), np.array(x_test), np.array(y_train), np.array(y_test)

    return x_train, x_test, y_train, y_test, idx_train, idx_test


x_train, x_test, y_train, y_test, idx_train, idx_test = train_test_split_default(bags, labels, idx)

The number of generated pmapper descriptors may be quite large, which can hinder model training. A representative set of descriptors can be selected by removing redundant descriptors with rare occurrences. Namely, descriptors with non-zero values in less than N % of the training conformations are removed.

In [6]:
from sklearn.preprocessing import MinMaxScaler

def remove_dsc(bags, tresh_down=0.1, tresh_up=1):
    
    bags_concat = np.concatenate(bags)
    
    tresh_down = tresh_down * len(bags_concat)
    tresh_up = tresh_up * len(bags_concat)
    
    out = []
    for dsc in range(bags_concat.shape[-1]):
        p = sum(np.where(bags_concat[:, dsc] == 0, 0, 1))
        if p < tresh_down or p > tresh_up:
            out.append(dsc)
    bags = [np.delete(bag, out, axis=1) for bag in bags]
    return out, np.array(bags)

def scale_data(x_train, x_test):
    scaler = MinMaxScaler()
    scaler.fit(np.vstack(x_train))
    x_train_scaled = x_train.copy()
    x_test_scaled = x_test.copy()
    for i, bag in enumerate(x_train):
        x_train_scaled[i] = scaler.transform(bag)
    for i, bag in enumerate(x_test):
        x_test_scaled[i] = scaler.transform(bag)
    return np.array(x_train_scaled), np.array(x_test_scaled)


out_dsc, x_train_selected = remove_dsc(x_train, tresh_down=0.1, tresh_up=1) 
x_test_selected = np.array([np.delete(x, out_dsc, axis=1) for x in x_test])

x_train_scaled, x_test_scaled = scale_data(x_train_selected, x_test_selected)

Models were developed with a multi-instance neural network with an attention mechanism , which highlights a few reactive conformations, responsible for observed selectivity and ignores the irrelevant conformations introducing noise in the modeling process. Namely, the attention mechanism assigns each conformation a weight from 0 to 1, determining its importance in terms of predicting catalyst selectivity. The sum of all attention weights equals 1.  
In learning process, each instance (conformation descriptor vector) runs through three fully-connected layers with 256, 128, and 64 hidden neurons (**ndim** parameter). Then the learned instance representations inputs to the attention network with 64 hidden neurons (**det_ndim**) and the number of output neurons equal to the number of input instances. The output neurons are followed by a Softmax unit, calculating attention weights for each instance. The learned instance representations are averaged considering the attention weights, resulting in the embedding vector, which is used to predict selectivity.
<img src="img/attention_net.png" width="500"/>

One should implement a protocol for optimizing the hyperparameters of the neural network model. Here we assign the optimal hyperparameters found with the *hyperopt* package.

In [8]:
from miqssr.estimators.attention_nets import AttentionNetRegressor

ndim = (x_train_selected[0].shape[1], 256, 128, 64) # number of hidden layers and neurons in the main network
det_ndim = (64,)                                    # number of hidden layers and neurons in the attention network
n_epoch = 100                                       # maximum number of learning epochs
lr = 0.001                                          # learning rate
weight_decay = 0.1                                  # l2 regularization
att_weight_dropout = 0.9                            # attention weights regularization
batch_size = 64                                     # batch size
init_cuda = True                                    # True if GPU is available


net = AttentionNetRegressor(ndim=ndim, det_ndim=det_ndim, init_cuda=init_cuda)
net.fit(x_train_selected, y_train, 
        n_epoch=n_epoch, 
        lr=lr,
        weight_decay=weight_decay,
        dropout=att_weight_dropout,
        batch_size=batch_size)

/home/zankov/catalyst/3D-MIL-QSSR/miqssr/estimators/attention_nets.py:23: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629431274/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  d1 = [i[0].nonzero().flatten().tolist() for i in w_new]


In [11]:
from sklearn.metrics import r2_score, mean_absolute_error

y_pred = net.predict(x_test_selected)

print(f'Determination coefficient (test set): {r2_score(y_test, y_pred):.2f}')
print(f'ΔΔG Mean absolute error (test set): {mean_absolute_error(y_test, y_pred):.2f} kcal/mol')

Determination coefficient (test set): 0.81
ΔΔG Mean absolute error (test set): 0.23 kcal/mol
